## README: CMIP6-LE and CMIP5-LE Processing Pipeline for Calculating Hemispheric Total Sea-Ice Area
The code here can be used to merge .nc files based on time (1850-1950 + 1950-2014), combine ensemble members (r1,r2,...,rn) into a single .nc file, and calculate siarea in the northern and southern hemisphere. A lot of the code is copy and pasted between cells, for different models and edge cases. TODO: When I have some extra time I need to pull out repeated code into multiple purpose functions.

For info on Large-Ensembles refer to: https://www.cesm.ucar.edu/projects/community-projects/MMLEA/ (outdated)

In [7]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import cmocean as cmo
import xarray as xr
import os
import re
from glob import glob
from collections import defaultdict

# CDO warnings annoy me...
import warnings
warnings.filterwarnings("ignore")

os.system(f"module load cdo")

0

In [28]:
models = [
    # "ACCESS-ESM1-5",
    # "ACCESS-CM2",
    "CanESM5", 
    # "CNRM-CM6-1",
    # "FGOALS-g3",  # TODO
    # "IPSL-CM6A-LR", #TODO
    # "MPI-ESM1-2-LR",
    # "MPI-ESM1-2-HR", 
    # "NorESM2-LM",
    # "MIROC-ES2L",
    # "MIROC6", 
]

meta = {
    "historical": defaultdict(lambda: {}), 
    "ssp126": defaultdict(lambda: {}),
    "ssp245": defaultdict(lambda: {}),
    "ssp370": defaultdict(lambda: {}),
    "ssp585": defaultdict(lambda: {})
}

#### Utils

In [29]:
def get_ensmem(dfiles):
# Get ensemble members:
    ensmem = []
    for i in range(len(dfiles)):
        s = re.search('(.*_r)(.*)(.*i1)', dfiles[i])
        if s == None: continue 
        ensmem.append(s.group(2))
    ensmem = np.unique(ensmem)
    return ensmem, len(ensmem) 

def drop_duplicates(ds): 
    return xr.DataArray.drop_duplicates(ds, dim='time')

# CMIP6 Processing Pipeline

### Step 1: Combine files that do not span entire time-period

In [5]:
for exp in meta.keys():
    droot = "/glade/scratch/zespinosa/cmip6/scenarios/raw_data/data/"
    time = "201501-210012.nc"
    if exp == "historical": 
        droot = "/glade/scratch/zespinosa/cmip6/historical/raw_data/data/"
        time = "185001-201412.nc"
    
    for model in models:
        print(f"Begin Processing {exp} {model}")
        try:  
            fp = os.path.join(droot, f"v_siconc_f_mon_e_{exp}_s_{model}")

            # Get ensemble members: 
            dfiles = sorted(glob(os.path.join(fp, '*.nc')))
            if len(dfiles) != 0:
                ensmem, nensmem = get_ensmem(dfiles)
                meta[exp][model]["ensmem"] = ensmem
                meta[exp][model]["nensmem"] = nensmem
                print("Ensemble Members: ", ensmem)

                # Combine files that do not span entire time-period and/or move into 'combined' folder
                dest = os.path.join(fp, f"{exp}_combined")
                if not os.path.exists(dest):
                    os.mkdir(dest)
                date = dfiles[0].split("_")[-1]
                if len(os.listdir(dest)) == 0:
                    if date != time:
                        for i in ensmem: 
                            sfiles = os.path.join(fp, f"*_{model}_{exp}_r{i}i1p1f2_gn*.nc")
                            sp = os.path.join(dest,f"siconc_SImon_{model}_{exp}_r{i}i1p1f2_gn_{time}")
                            print(f"Creating {sp}")
                            os.system(f"module load cdo; cdo -s mergetime {sfiles} {sp}")
                    else: 
                        print(f"Moving files to {dest}")
                        for f in dfiles: 
                            os.system(f"mv {f} {dest}")
        except Exception as err:
            print(f"{model} Failed! \n {err}")

Begin Processing historical MIROC-ES2L
Begin Processing historical MIROC6
Ensemble Members:  ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35' '36'
 '37' '38' '39' '4' '40' '41' '42' '43' '44' '45' '46' '47' '48' '49' '5'
 '50' '6' '7' '8' '9']
Begin Processing ssp126 MIROC-ES2L
Begin Processing ssp126 MIROC6
Begin Processing ssp245 MIROC-ES2L
Begin Processing ssp245 MIROC6
Ensemble Members:  ['1' '10' '11' '12' '13' '14' '15' '16' '17' '18' '19' '2' '20' '21' '22'
 '23' '24' '25' '26' '27' '28' '29' '3' '30' '31' '32' '33' '34' '35' '36'
 '37' '38' '39' '4' '40' '41' '42' '43' '44' '45' '46' '47' '48' '49' '5'
 '50' '6' '7' '8' '9']
Begin Processing ssp370 MIROC-ES2L
Begin Processing ssp370 MIROC6
Begin Processing ssp585 MIROC-ES2L
Begin Processing ssp585 MIROC6


### Step 2: Calculate SIA for historical and scenarios and combine based on ensemble members

In [31]:
for model in models: 
    # try: 
        print(f"Begin processing for model: {model} \n")

        rootHist = f"/glade/scratch/zespinosa/cmip6LE/historical/raw_data/data/v_siconc_f_mon_e_historical_s_{model}/historical_combined/" 
        histFiles = sorted(glob(rootHist + '/*p1f1_gn*.nc'))

        print(f"  # historical ensemble members = {len(histFiles)}")

        # Get ensemble members
        ensHist, nensHist = get_ensmem(histFiles)

        # Load historical data
        mfds = xr.open_mfdataset(histFiles, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
        mfds = mfds.assign_coords({'ensmem': ('ensmem', np.arange(0, len(histFiles)))})

        # Get grid cell area file
        ddir = '/glade/scratch/zespinosa/cmip6LE/historical/area/data/'
        dfile = sorted(glob(os.path.join(ddir, "areacello*"+model+"*.nc")))[0]
        print(f"  Area filename: {dfile}")
        ncf = os.path.join(ddir, dfile)
        ds = xr.open_dataset(ncf)

        area = ds['areacello']
        siconc = mfds['siconc']/100

        # Compute sea ice area in each grid cell
        siarea = siconc*area.data[None, None, :, :]
        siarea = siarea.rename('siarea')

        # Get name of lat and lon dimensions, these differ by model
        spatial_dims = siarea.dims[-2:]
        print(f"  Spatial dimension names: {spatial_dims}")

        # Get index of approximate middle of latitudes to separate Arctic and Antarctic
        lat_mid_index = int(siarea.shape[2]/2)
        print(f"  Index of middle latitude (rounded): {lat_mid_index}")

        # Get first latitude to determine if lats start in Arctic or Antarctic
        # First need to get name of latitude variable, differs by model
        possible_latnames = ["lat", "latitude", "nav_lat"]
        for name in possible_latnames:
            if name in ds.coords:
                latname = name

        lat0 = siarea[latname][0].compute()[0]

        print(f"  Latitudes start from: {lat0.data}")
        print("  Beginning historical siarea calculation . . .")

        if lat0 < 0: # If latitudes start from Antarctica
            siarea_nh_hist = siarea[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
            siarea_sh_hist = siarea[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
        else: # If latitudes start from Arctic
            siarea_nh_hist = siarea[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
            siarea_sh_hist = siarea[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()

        print("  Done with historical siarea calculation ")

        for exp in meta.keys():
            dest = "/glade/scratch/zespinosa/cmip6LE/processed"
            dout_nh = f'siarea_nh_historical_{exp}_'+model+'i1p1f1.nc'
            dout_sh = f'siarea_sh_historical_{exp}_'+model+'i1p1f1.nc'
            output_nh = os.path.join(dest,dout_nh)
            output_sh = os.path.join(dest,dout_sh)
            
            if os.path.exists(output_nh) and os.path.exists(output_sh): continue
            if exp == "historical": continue

            rootScen = f"/glade/scratch/zespinosa/cmip6LE/scenarios/raw_data/data/v_siconc_f_mon_e_{exp}_s_{model}/{exp}_combined/" 
            scenFiles = sorted(glob(rootScen + '/*p1f1_gn*.nc'))

            # Get ensemble members
            ensScen, nensScen = get_ensmem(scenFiles)
            print(f"  # {exp} ensemble members = {nensScen}")


            # Load scenario data
            mfdsScen = xr.open_mfdataset(scenFiles, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
            mfdsScen = mfdsScen.assign_coords({'ensmem': ('ensmem', np.arange(0, len(scenFiles)))})

            siconcScen = mfdsScen['siconc']/100

            # Compute sea ice area in each grid cell
            siareaScen = siconcScen*area.data[None, None, :, :]
            siareaScen = siareaScen.rename('siarea')

            print(f"  Beginning {exp} siarea calculation . . .")

            if lat0 < 0: # If latitudes start from Antarctica
                siarea_nh_scen = siareaScen[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
                siarea_sh_scen = siareaScen[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
            else: # If latitudes start from Arctic
                siarea_nh_scen = siareaScen[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
                siarea_sh_scen = siareaScen[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()

            print(f"  Done with {exp} siarea calculation ")

            # Get overlapping ensemble members of scenario and historical
            overlap = list(set(ensScen) & set(ensHist))
            histMembers = [list(ensHist).index(i) for i in overlap]
            scenMembers = [list(ensScen).index(i) for i in overlap]
            
            print(f"  # overlapping ensemble members = {len(overlap)}")

            siarea_nh = xr.concat([siarea_nh_hist[histMembers,:],siarea_nh_scen[scenMembers,:]], join="override", dim="time")
            siarea_sh = xr.concat([siarea_sh_hist[histMembers,:],siarea_sh_scen[scenMembers,:]], join="override", dim="time")

                
            print("  Saving to cdf . . .")
            print(f"  Arctic filename historical {sp}: {dout_nh}")            
            siarea_nh.to_dataset().to_netcdf(output_nh)
            print(f"  Antarctic filename historical {sp}: {dout_sh}")
            siarea_sh.to_dataset().to_netcdf(output_sh)
    
    # except Exception as err:
        # print(f"{model} Failed! \n {err}")
            
        print(f"  Done with {model}\n")

Begin processing for model: CanESM5 

  # historical ensemble members = 25
  Area filename: /glade/scratch/zespinosa/cmip6LE/historical/area/data/areacello_Ofx_CanESM5_historical_r1i1p1f1_gn.nc
  Spatial dimension names: ('j', 'i')
  Index of middle latitude (rounded): 145
  Latitudes start from: -78.39350128173828
  Beginning historical siarea calculation . . .
  Done with historical siarea calculation 
  # ssp126 ensemble members = 25
  Beginning ssp126 siarea calculation . . .
  Done with ssp126 siarea calculation 
  # overlapping ensemble members = 25
  Saving to cdf . . .
  Arctic filename historical ssp126: siarea_nh_historical_ssp126_CanESM5i1p1f1.nc
  Antarctic filename historical ssp126: siarea_sh_historical_ssp126_CanESM5i1p1f1.nc
  # ssp245 ensemble members = 25
  Beginning ssp245 siarea calculation . . .
  Done with ssp245 siarea calculation 
  # overlapping ensemble members = 25
  Saving to cdf . . .
  Arctic filename historical ssp245: siarea_nh_historical_ssp245_CanESM5i

# CESM2-LE Processing from Casper

In [10]:
droot = "/glade/scratch/zespinosa/cmip6/CESM2/"

# Download historical files
print(f"  Load historical CESM2 Files")
dFiles = sorted(glob(droot + 'siarea_aice_BHISTcmip6*_CESM2.nc'))
mfdsHist = xr.open_mfdataset(dFiles, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
mfdsHist = mfdsHist.assign_coords({'ensmem': ('ensmem', np.arange(0, len(dFiles)))})

# Download scenario files
print(f"  Load SSP370 CESM2 Files")
dFiles = sorted(glob(droot + 'siarea_aice_BSSP370cmip6*_CESM2.nc'))
mfdsSSP370 = xr.open_mfdataset(dFiles, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
mfdsSSP370 = mfdsSSP370.assign_coords({'ensmem': ('ensmem', np.arange(0, len(dFiles)))})

aiceHist = mfdsHist['aice']
aiceSSP370 = mfdsSSP370['aice']

# Get grid cell area file
ncf = sorted(glob(os.path.join(droot, "areacello*.nc")))[0]
print(f"  Area filename: {ncf}")
ds = xr.open_dataset(ncf)

area = ds['areacello']

# Get name of lat and lon dimensions, these differ by model
spatial_dims = aiceHist.dims[-2:]
print(f"  Spatial dimension names: {spatial_dims}")

# Get index of approximate middle of latitudes to separate Arctic and Antarctic
lat_mid_index = int(aiceHist.shape[2]/2)
print(f"  Index of middle latitude (rounded): {lat_mid_index}")

# Get first latitude to determine if lats start in Arctic or Antarctic
# First need to get name of latitude variable, differs by model
lat0 = aiceHist['TLAT'][0, 0].compute()

print(f"  Latitudes start from: {lat0.data}")
print("  Beginning siarea calculation . . .")

# Compute sea ice area in each grid cell
siareaHist = aiceHist*area.data[None, None, :, :]
siareaHist = siareaHist.rename('siarea')

siareaSSP370 = aiceSSP370*area.data[None, None, :, :]
siareaSSP370 = siareaSSP370.rename('siarea')

if lat0 < 0: # If latitudes start from Antarctica
    siarea_nh_ssp370 = siareaSSP370[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
    siarea_sh_ssp370 = siareaSSP370[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
    siarea_nh_hist = siareaHist[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
    siarea_sh_hist = siareaHist[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
else: # If latitudes start from Arctic
    siarea_nh_ssp370 = siareaSSP370[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
    siarea_sh_ssp370 = siareaSSP370[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
    siarea_nh_hist = siareaHist[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
    siarea_sh_hist = siareaHist[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()

print(f"  Done with siarea calculation ")

siarea_nh = xr.concat([siarea_nh_hist,siarea_nh_ssp370], join="override", dim="time")
siarea_sh = xr.concat([siarea_sh_hist,siarea_sh_ssp370], join="override", dim="time")

sp = "/glade/scratch/zespinosa/cmip6/processed"
dout_nh = os.path.join(sp, "siarea_nh_historical_ssp370_CESM2.nc")
dout_sh = os.path.join(sp, "siarea_sh_historical_ssp370_CESM2.nc")
print("  Saving to cdf . . .")

print(f"  Arctic filename historical: {dout_nh}")            
siarea_nh.to_dataset().to_netcdf(dout_nh)
print(f"  Antarctic filename historical: {dout_sh}")
siarea_sh.to_dataset().to_netcdf(dout_sh)
print("Done!") 

  Load historical CESM2 Files
  Load SSP370 CESM2 Files
  Area filename: /glade/scratch/zespinosa/cmip6/CESM2/areacello_CESM2.nc
  Spatial dimension names: ('nj', 'ni')
  Index of middle latitude (rounded): 192
  Latitudes start from: -79.22052001953125
  Beginning siarea calculation . . .
  Done with siarea calculation 
  Saving to cdf . . .
  Arctic filename historical: /glade/scratch/zespinosa/cmip6/processed/siarea_nh_historical_ssp370_CESM2.nc
  Antarctic filename historical: /glade/scratch/zespinosa/cmip6/processed/siarea_sh_historical_ssp370_CESM2.nc
Done!


# CMIP5-LE
Process code at '/glade/collections/cdg/data/CLIVAR_LE' 

In [16]:
modelsCMIP5 = {
    # "CSIRO-Mk3-6-0": "csiro_mk36_lens",     # NEED areacello  (96x192)
    # "CanESM2": "canesm2_lens",              # DONE
    # "GFDL-CM3": "gfdl_cm3_lens",            # DONE
    # "GFDL-ESM2M": "gfdl_esm2m_lens",        # NEED areacello (180x360)
    # "CESM1-CAM5": "cesm_lens",                # DONE
    # "MPI-ESM": "mpi_lens",                  # Combine Historic and RCP (good with areacello)
}
exps = ["rcp85"]

In [17]:
droot = "/glade/collections/cdg/data/CLIVAR_LE"
dsave = "/glade/scratch/zespinosa/cmip6LE/processed"

for model, mpath in modelsCMIP5.items():
    for exp in exps: 
        try: 
            # Load Datafiles
            dFiles = sorted(glob(os.path.join(droot, mpath, "OImon", "sic", f"*{model}*{exp}*192001-210012.nc")))
            print(f"  Load {model} {exp} files {len(dFiles)}")
            mfds = xr.open_mfdataset(dFiles, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
            # import pdb; pdb.set_trace()
            
#             dFilesHist = sorted(glob(os.path.join(droot, mpath, "OImon", "sic", f"*{model}*historical*.nc")))
#             print(f"  Load {model} hist files {len(dFilesHist)}")
#             mfdsHist = xr.open_mfdataset(dFilesHist, combine='nested', concat_dim='ensmem', preprocess=drop_duplicates)
            
#             mfds = xr.concat([mfdsHist,mfdsRCP], join="override", dim="time")
            
            # Get grid cell area file
            # ncf = "/glade/scratch/zespinosa/cmip5/area/areacello_CanESM2.nc"
            ncf = sorted(glob(os.path.join(droot, mpath, "fx", "areacello", "*areacello*.nc")))[0]
            print(f"  Area filename: {ncf}")
            ds = xr.open_dataset(ncf)
            
            aice = mfds['sic']/100
            area = ds['areacello']

            # Get name of lat and lon dimensions, these differ by model
            spatial_dims = aice.dims[-2:]
            print(f"  Spatial dimension names: {spatial_dims}")

            # Get index of approximate middle of latitudes to separate Arctic and Antarctic
            lat_mid_index = int(aice.shape[2]/2)
            print(f"  Index of middle latitude (rounded): {lat_mid_index}")

            # Get first latitude to determine if lats start in Arctic or Antarctic
            # First need to get name of latitude variable, differs by model
            possible_latnames = ["j","rlat", "lat", "latitude", "nav_lat"]
            for name in possible_latnames:
                if name in ds.coords:
                    latname = name
                    break

            lat0 = aice[latname][0].compute()

            print(f"  Latitudes start from: {lat0.data}")
            print("  Beginning siarea calculation . . .")

            # Compute sea ice area in each grid cell
            siarea = aice*area.data[None, None, :, :]
            siarea = siarea.rename('siarea')

            if lat0 < 0: # If latitudes start from Antarctica
                siarea_nh = siarea[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()
                siarea_sh = siarea[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
            else: # If latitudes start from Arctic
                siarea_nh = siarea[:, :, :lat_mid_index, :].sum(spatial_dims) #.compute()
                siarea_sh = siarea[:, :, lat_mid_index:, :].sum(spatial_dims) #.compute()

            time = dFiles[0].split("_")[-1]
            dout_nh = os.path.join(dsave, f"siarea_nh_historical_{exp}_{model}_192001-210012.nc")
            dout_sh = os.path.join(dsave, f"siarea_sh_historical_{exp}_{model}_192001-210012.nc")
            
            print("  Saving to cdf . . .")
            print(f"  Arctic filename historical: {dout_nh}")            
            siarea_nh.to_dataset().to_netcdf(dout_nh)
            print(f"  Antarctic filename historical: {dout_sh}")
            siarea_sh.to_dataset().to_netcdf(dout_sh)
            print("Done!") 

        except Exception as err:
            print(f"{model} Failed! \n {err}")
    

  Load GFDL-CM3 rcp85 files 20
  Area filename: /glade/collections/cdg/data/CLIVAR_LE/gfdl_cm3_lens/fx/areacello/areacello_fx_GFDL-CM3_historical_r0i0p0.nc
  Spatial dimension names: ('rlat', 'rlon')
  Index of middle latitude (rounded): 100
  Latitudes start from: -81.5
  Beginning siarea calculation . . .
  Saving to cdf . . .
  Arctic filename historical: /glade/scratch/zespinosa/cmip6LE/processed/siarea_nh_historical_rcp85_GFDL-CM3_192001-210012.nc
  Antarctic filename historical: /glade/scratch/zespinosa/cmip6LE/processed/siarea_sh_historical_rcp85_GFDL-CM3_192001-210012.nc
Done!
